In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# KROK 1: WSTĘPNA ANALIZA

In [2]:
# Zaimortowanie bazy danych
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
df = pd.read_csv(url, header=None)
column_names = ['class', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols',
                'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue',
                'diluted_wines', 'proline']
df.columns = column_names
df.head()

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,diluted_wines,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


# KROK 2: PRZYGOTOWANIE DANYCH

In [3]:
# Podział danych na features i target
X = df.drop('class', axis=1)
y = df['class']

In [4]:
# Standaryzacja danych
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X = MinMaxScaler().fit_transform(X)

In [5]:
# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)


# KROK 3: STWORZENIE MODELU OPARTEGO NA XGBOOST

In [ ]:
xgb_cl = xgb.XGBClassifier()

# KROK 4: TRENING I WALIDACJA

In [6]:
# Trenowanie modelu
xgb_cl.fit(X_train, y_train)

# Przewidywanie na zbiorze testowym
y_test_preds_xgb = xgb_cl.predict(X_test)

# Ocena wyników
print("XGBoost Classification Report:")
print(classification_report(y_test, y_test_preds_xgb))

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       1.00      0.92      0.96        13
           2       0.82      1.00      0.90         9

    accuracy                           0.94        36
   macro avg       0.94      0.95      0.94        36
weighted avg       0.95      0.94      0.95        36



In [26]:
accuracy_score(y_test, y_test_preds_xgb)

0.9444444444444444

# KROK 5: OPTYMALIZACJA PARAMETRÓW

In [7]:
param_grid = {
    "max_depth": [2, 3, 4, 5, 7, 9, 15],
    "learning_rate": [0.1, 0.01, 0.05, 0.2, 0.5, 0.8],
    'n_estimators': [1, 4, 5, 10, 20, 40, 100],
}

In [10]:
# Init klasyfikator
xgb_cl = xgb.XGBClassifier(objective="reg:logistic")

# Init Grid Search
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="accuracy")

# Fit
_ = grid_cv.fit(X_train, y_train)

In [11]:
grid_cv.best_score_

0.978871158392435

In [ ]:
# na podstawie 13 features model w 98% przewiduje do jakiej z 3 klas należy wino

In [12]:
grid_cv.best_params_

{'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 20}

In [13]:
grid_cv.best_estimator_.feature_importances_

array([1.7579518e-02, 4.3910369e-02, 7.3612062e-03, 4.3159877e-03,
       8.9083038e-02, 7.4833422e-04, 1.4431599e-01, 1.3253583e-03,
       3.3235323e-04, 1.5412460e-01, 1.9095754e-02, 3.6425728e-01,
       1.5355018e-01], dtype=float32)

# KROK 6: ULEPSZENIE MODELU

Nasz model osiągnął najlepsze wyniki przy następujących parametrach 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 20

### PODSUMOWANIE

**Prosta sieć neuronowa (MODEL 1):**
- Dokładność (accuracy) na zbiorze testowym wynosi 1.0000 (100%).
- W raporcie klasyfikacji (classification report) widać, że dla każdej klasy uzyskano 100% precision, recall i F1-score.
- Wartość straty (loss) na zbiorze testowym wynosi 0.0499.

**Model XGBoost (MODEL 2):**
- Dokładność (accuracy_score) na zbiorze testowym wynosi 0.9444 (94.44%).
- Najlepszy wynik uzyskany podczas przeszukiwania siatki (grid search) wynosi 0.9789 (97.89%).
- W raporcie klasyfikacji widać, że dla klasy 0, precision wynosi 1.00, recall wynosi 0.93, a F1-score wynosi 0.96. Dla klasy 1, precision wynosi 1.00, recall wynosi 0.92, a F1-score wynosi 0.96. Dla klasy 2, precision wynosi 0.82, recall wynosi 1.00, a F1-score wynosi 0.90.

**Porównanie obu modeli:**

**Model 1 (Prosta sieć neuronowa)** 
osiąga dokładność 100%, co sugeruje, że doskonale radzi sobie z klasyfikacją. Precision, recall i F1-score wynoszą 100% dla wszystkich klas, co świadczy o pełnej poprawności predykcji.

**Model 2 (XGBoost)** 
ma nieco niższą dokładność na poziomie 94.44% na zbiorze testowym. Jednak podczas przeszukiwania siatki, osiągnął najlepszy wynik na poziomie 97.89%, co wskazuje, że przy optymalnie dobranych parametrach, może dawać bardzo dobre rezultaty.

**Wnioski:**

**Model 1 (Prosta sieć neuronowa)** 
osiąga wyższą dokładność i idealne wyniki dla wszystkich metryk klasyfikacji. Wydaje się być lepszy od Modelu 2 (XGBoost) na podstawie osiągniętych wyników. Wymaga mniejszego wkładu pracy i osiąga lepsze wyniki.

**Model 2 (XGBoost)** 
może być bardziej złożony do konfiguracji, ale z odpowiednim dostrojeniem parametrów, może uzyskać bardzo dobre wyniki. 